# Genetic Algorithm

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import random

from sklearn.model_selection import train_test_split

# yahoo finance is used to fetch data 
import yfinance as yf
yf.pdr_override()

In [2]:
# input
symbol = 'AMD'
start = '2014-01-01'
end = '2019-01-01'

# Read data 
dataset = yf.download(symbol,start,end)

[*********************100%***********************]  1 of 1 completed


In [3]:
dataset['Open_Close'] = (dataset['Open'] - dataset['Adj Close'])/dataset['Open']
dataset['High_Low'] = (dataset['High'] - dataset['Low'])/dataset['Low']
dataset['Increase_Decrease'] = np.where(dataset['Volume'].shift(-1) > dataset['Volume'],1,0)
dataset['Buy_Sell_on_Open'] = np.where(dataset['Open'].shift(-1) > dataset['Open'],1,0)
dataset['Buy_Sell'] = np.where(dataset['Adj Close'].shift(-1) > dataset['Adj Close'],1,0)
dataset['Returns'] = dataset['Adj Close'].pct_change()
dataset = dataset.dropna()
dataset.head()

# View Columns
dataset.head()

,Open,High,Low,Close,Adj Close,Volume,Open_Close,High_Low,Increase_Decrease,Buy_Sell_on_Open,Buy_Sell,Returns
Date,,,,,,,,,,,,
2014-01-03,3.98,4.00,3.88,4.00,4.00,22887200,-0.005025,0.030928,1,1,1,0.012658
2014-01-06,4.01,4.18,3.99,4.13,4.13,42398300,-0.029925,0.047619,1,1,1,0.032500
2014-01-07,4.19,4.25,4.11,4.18,4.18,42932100,0.002387,0.034063,0,1,0,0.012106
2014-01-08,4.23,4.26,4.14,4.18,4.18,30678700,0.011820,0.028986,0,0,0,0.000000
2014-01-09,4.20,4.23,4.05,4.09,4.09,30667600,0.026190,0.044444,0,0,1,-0.021531


In [4]:
X = np.array(dataset['Open'].values)
y = dataset['Increase_Decrease'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                   test_size=0.4, random_state=42)

In [5]:
# Define the fitness function (e.g., a simple moving average crossover strategy)
def fitness_function(strategy, X_train, X_test, y_train, y_test):
    # Implement your trading strategy and calculate profit or loss
    # Example: Buy when short-term MA > long-term MA, and sell when the opposite is true
    profit = 0
    for i in range(1, len(X_test)):
        if strategy[i] > strategy[i - 1] and y_test[i] == 1:
            profit += X_test[i] - X_test[i - 1]
        elif strategy[i] < strategy[i - 1] and y_test[i] == 0:
            profit += X_test[i - 1] - X_test[i]
    return profit

# Create an initial population
def initialize_population(pop_size, strategy_length):
    population = []
    for _ in range(pop_size):
        strategy = [random.randint(0, 1) for _ in range(strategy_length)]
        population.append(strategy)
    return population

# Genetic algorithm
def genetic_algorithm(X_train, X_test, y_train, y_test, pop_size, generations):
    strategy_length = len(X_test)  # Length of the trading strategy
    population = initialize_population(pop_size, strategy_length)

    for generation in range(generations):
        # Evaluate fitness of the population
        fitness_scores = [fitness_function(strategy, X_train, X_test, y_train, y_test) for strategy in population]

        # Select the top-performing individuals
        selected_population = []
        for _ in range(pop_size):
            idx1, idx2 = random.sample(range(pop_size), 2)
            if fitness_scores[idx1] > fitness_scores[idx2]:
                selected_population.append(population[idx1])
            else:
                selected_population.append(population[idx2])

        # Crossover
        new_population = []
        for i in range(0, pop_size, 2):
            parent1 = selected_population[i]
            parent2 = selected_population[i + 1]
            crossover_point = random.randint(1, strategy_length - 1)
            child1 = parent1[:crossover_point] + parent2[crossover_point:]
            child2 = parent2[:crossover_point] + parent1[crossover_point:]
            new_population.extend([child1, child2])

        # Mutation
        mutation_rate = 0.01  # Adjust as needed
        for i in range(pop_size):
            if random.random() < mutation_rate:
                mutation_point = random.randint(0, strategy_length - 1)
                population[i][mutation_point] = 1 - population[i][mutation_point]

        population = new_population

    # Find the best-performing individual
    best_strategy = max(population, key=lambda x: fitness_function(x, X_train, X_test, y_train, y_test))
    return best_strategy

In [6]:
# Example usage
best_strategy = genetic_algorithm(X_train, X_test, y_train, y_test, pop_size=100, generations=100)
print("Best strategy:", best_strategy)

Best strategy: [0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 